# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096-iap17/blob/master/lecture1/Boxes-and-registers.ipynb.)

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes

$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i.
$$

In [1]:
a = rand(10^7) # array of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.130461 
 0.821782 
 0.660476 
 0.99516  
 0.355917 
 0.418669 
 0.279209 
 0.829117 
 0.536267 
 0.259042 
 0.471322 
 0.81795  
 0.202128 
 ⋮        
 0.0897454
 0.381729 
 0.0583961
 0.0460956
 0.971274 
 0.758764 
 0.789532 
 0.238554 
 0.113996 
 0.883042 
 0.965278 
 0.386773 

In [2]:
sum(a) # one expects this is 10^7 * .5 , since the mean of each entry is .5

4.998789120724919e6

# Benchmarking a few implementations in different languages

In [3]:
using BenchmarkTools: @btime, @benchmark

# BenchmarkTools provides the @benchmark and @btime macros,
# which run a given function multiple times to provide a 
# robust estimate of its true runtime. 

#  1. The C language: (8.0 msecs)

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

You can put C code in a Julia session, compile it, and run it.

In [4]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    size_t i;
    for (i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function implemnted above:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [5]:
c_sum(a)

4.998789120724622e6

In [6]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbol

true

We can now benchmark the C code directly from Julia:

In [7]:
@btime c_sum($a)

  8.238 ms (0 allocations: 0 bytes)


4.998789120724622e6

# 2. Python's built in `sum` (133 msecs)

In [8]:
# Julia interface to Python:
using PyCall

In [9]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [10]:
pysum(a)

4.998789120724622e6

In [11]:
pysum(a) ≈ sum(a)

true

In [12]:
@btime $pysum($apy_list)

  137.670 ms (17 allocations: 512 bytes)


4.998789120724622e6

# 3. Python: `numpy` (4.5 msec)  

## Takes advantage of hardware "SIMD"

`numpy` is an optimized C library, callable from Python

If it is not installed, install it from Julia as follows:

In [13]:
# using Conda 
# Conda.add("numpy")

In [14]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

numpy_sum(apy_list)

  4.869 ms (17 allocations: 512 bytes)


4.998789120724917e6

In [16]:
numpy_sum(apy_list) ≈ sum(a)

true

In [ ]:
@btime $numpy_sum($apy_numpy)

# 4. Python, hand written (353 msec!)

In [17]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
mysum_py = py"""mysum"""o

PyObject <function mysum at 0x7f45f2cf4cf8>

In [19]:
mysum_py(apy_list)

4.998789120724622e6

In [20]:
mysum_py(apy_list) ≈ sum(a)

true

In [18]:
@btime $mysum_py($apy_list)

  388.884 ms (17 allocations: 512 bytes)


4.998789120724622e6

# 5. Julia (built-in) (4.4 msec) 

## Written directly in Julia, not in C! 

(and just as fast as numpy's optimized `sum()`)

In [21]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [22]:
@btime sum($a)

  4.623 ms (0 allocations: 0 bytes)


4.998789120724919e6

# 6. Julia (hand-written) (8.0 msec, same as hand-written C)

In [23]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [24]:
@btime mysum($a)

  8.296 ms (0 allocations: 0 bytes)


4.998789120724622e6

## Conclusion

It's often pretty straightforward to write Julia code which is as fast as well-written C code! Loops in Julia are fast, so feel free to use them whenever they help organize or clarify your code.